In [ ]:
!pip install perfplot
!pip install --upgrade "jax[cpu]"

In [ ]:
# NVIDIA GPU JAX Installation
!pip install --upgrade pip
# CUDA 12 installation
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [ ]:
import numpy as np
import jax 
import jax.numpy as jnp
import torch

In [ ]:
NUM_THREADS = 12
torch.set_num_threads(NUM_THREADS)

In [ ]:
import perfplot

perfplot.show(
    setup=lambda n: [np.random.rand(n, n), np.random.rand(n,n), torch.rand(n,n), torch.rand(n,n)] ,
    kernels=[
        lambda data: np.dot( data[0], data[1]),
        lambda data: jnp.dot( data[0], data[1]),
        lambda data: torch.matmul( data[2], data[3]),
    ],
    title="Matrix multiplication",
    labels=["Numpy", "Jax", "Torch"],
    n_range=[2 ** k for k in range(15)],
    xlabel="len(a)",
    equality_check=False,
)

In [ ]:
import perfplot

perfplot.show(
    setup=lambda n: [np.random.rand(n, n), torch.rand(n,n)] ,
    kernels=[
        lambda data: np.exp( data[0] ),
        lambda data: jnp.exp( data[0] ),
        lambda data: torch.exp( data[1] ),
    ],
    title="Element-wise exponentiation",
    labels=["Numpy", "Jax", "Torch"],
    n_range=[2 ** k for k in range(15)],
    xlabel="len(a)",
    equality_check=False,
)